# Generating a Master Data Dump

In [44]:
import os
import pandas as pd
import numpy as np
import re
import warnings
from collections import Counter
from itertools import chain

In [ ]:
def get_files_in_folder(folder):
    '''
    Generates a list of all of the Development Reports
    
    Args
    folder: the folder path (string) for the folder that contains the workbooks
    
    Outputs
    files: a list of all of the development reports
    '''
    files = []
    for file in os.listdir(folder):
        # Skip .DS_Store files
        if file == '.DS_Store':
            continue
        file_path = folder+'/'+file
        if os.path.isfile(file_path):
            files.append(file)
    return files

# Example
folder = os.getcwd()+'/data'
get_files_in_folder(folder)

['FY23 Development Report Workbook.xlsx',
 'FY24 Development Report Workbook.xlsx',
 'FY22 Development Report Workbook.xlsx',
 'FY25 Development Report Workbook.xlsx']

In [51]:
def get_file_fiscal_year(file):
    '''
    From the file path, finds the fiscal year of the development report workbook.
    
    Args
    file: the file path (string) for the development report workbook

    Outputs
    fiscal_year: the fiscal year (string) for the development report workbook
    '''
    # Get the fiscal year
    match = re.search(r'FY(\d{2})', file)
    if match:
        fiscal_year = '20' + match.group(1)
    else:
        # Issue a warning if the fiscal year is not found
        warnings.warn(f"The file name '{file}' does not specify a fiscal year. Please specify 'FY' in the file name.", UserWarning)
        fiscal_year = None  # Set to None to indicate missing fiscal year
    return fiscal_year

# Example
file = os.getcwd()+'/data/FY22 Development Report Workbook.xlsx'
get_file_fiscal_year(file)

'2022'

In [54]:
def build_sheet_details_df(file):
    ''' 
    Generate a sheet details data frame. 
    At this point, it's important to ensure that data dumps and five reports sheets are recognized out of the workbook.
    This is important because this data frame is later joined on the data dumps and/or five report data. 
    
    Args 
    file: the file path (string) for the development report workbook

    Outputs
    sheet_details_df: the details (DataFrame) of the workbook contents recognized by the program
    '''
    # Load the sheet names from the Excel file
    sheets = pd.ExcelFile(file).sheet_names
    
    # Isolate the data dump sheets while accounting for variations
    data_dump_sheets = sorted([
        sheet.replace("data dump", "Data Dump") 
        for sheet in sheets 
        if "data dump" in sheet.lower()
    ])

    # Isolate the five reports sheets while accounting for variations
    five_reports_sheets = sorted([
        re.sub(r'(\d{2}) 5 Reports', r'\1 Five Reports', sheet) 
        for sheet in sheets 
        if '5 Reports' in sheet or 'Five Reports' in sheet
    ])

    # Check if the number of Data Dump sheets matches Five Reports sheets
    if len(data_dump_sheets) != len(five_reports_sheets):
        raise ValueError("The number of 'Data Dump' sheets does not match the number of 'Five Reports' sheets. Please check the sheet names inside the file.")
    
    # Generate a list of the fiscal years for how many data dumps we have
    fiscal_year = get_file_fiscal_year(file)
    fiscal_years = [fiscal_year]*len(data_dump_sheets)
    
    # Extract the month numbers from the sheet names
    month_numbers = [re.match(r'(\d{2})', sheet).group(1) for sheet in data_dump_sheets if re.match(r'^\d{2} ', sheet)]
    
    # Create the DataFrame with the initial columns
    month_df_1 = pd.DataFrame({
        'Data Dump' : data_dump_sheets,
        'Five Reports' : five_reports_sheets,
        'Month Number' : month_numbers,  
        'Fiscal Year' : fiscal_years
    })
    
    # Populate the calendar year by subtracting 1 from the fiscal year if it is not one of the first 6 months of the year
    month_df_1['Calendar Year'] = month_df_1['Month Number'].apply(lambda x: str(int(fiscal_year)-1) if x not in ['01', '02', '03', '04', '05', '06'] else fiscal_year)

    # Generate a date that can be used in dashboards for time frame filters
    month_df_1['Date'] = pd.to_datetime(month_df_1['Calendar Year'].astype(str) + '-' + month_df_1['Month Number'] + '-01')

    # Create a dataframe of month names that we will join to month_df_1 so we have full month names
    month_df_2 = pd.DataFrame({
        'Month Number' : ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'],
        'Month' : ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
    })
    month_df_merged = pd.merge(month_df_1, month_df_2, how='left', left_on='Month Number', right_on='Month Number')
    
    # Rearrange the columns
    sheet_details_df = month_df_merged[['Data Dump', 'Five Reports', 'Date', 'Month Number', 'Month', 'Calendar Year', 'Fiscal Year']]

    return sheet_details_df

# Example
file = os.getcwd()+'/data/FY22 Development Report Workbook.xlsx'
build_sheet_details_df(file)

,Data Dump,Five Reports,Date,Month Number,Month,Calendar Year,Fiscal Year
0,01 Data Dump,01 Five Reports,2022-01-01,01,January,2022,2022
1,02 Data Dump,02 Five Reports,2022-02-01,02,February,2022,2022
2,03 Data Dump,03 Five Reports,2022-03-01,03,March,2022,2022
3,04 Data Dump,04 Five Reports,2022-04-01,04,April,2022,2022
4,05 Data Dump,05 Five Reports,2022-05-01,05,May,2022,2022
5,06 Data Dump,06 Five Reports,2022-06-01,06,June,2022,2022
6,07 Data Dump,07 Five Reports,2021-07-01,07,July,2021,2022
7,08 Data Dump,08 Five Reports,2021-08-01,08,August,2021,2022
8,09 Data Dump,09 Five Reports,2021-09-01,09,September,2021,2022
9,10 Data Dump,10 Five Reports,2021-10-01,10,October,2021,2022


# Extract a Data Dump from a Sheet

In [ ]:
# Sometimes the data dump and its revisions have column headers that aren't formatted the same.
# This mapping allows for the program to recognize column headers from data dumps.
# If data for a column is not showing up, the column header needs to be changed or added to fit the data requirements as seen in output data sets.

column_mapping = {
    "CATEGORY" : "Category",
    "RECORD NUMBER": 'Record Number',
    "REC_NO" : "Record Number",
    "REC_NO1" : "Record Number",
    "COMM_SUB" : "Commercial Subtype",
    "COMMERCIAL SUBTYPE" : "Commercial Subtype",
    "RES_SUB" : "Residential Subtype",
    "RESIDENTIAL SUBTYPE" : "Residential Subtype",
    "WORK DESCRIPTION": "Work Description",
    "WORK_DES1" : "Work Description",
    "WORK_DES" : "Work Description",
    "BUSINESS NAME" : "Business Name",
    "BUS_NAME1" : "Business Name",
    "BUS_NAME" : "Business Name",
    "PROJECT COST" : "Project Cost",
    "MUL_PROJ_COST" : "Project Cost",
    "JOB COST VALUATION" : "Job Cost Valuation",
    "MUL_JOB_COST_VAL" : "Job Cost Valuation",
    "CENSUS - CURRENT VALUATION RES" : "Current Valuation - Residential",
    "Current Valuation Res" : "Current Valuation - Residential",
    "CENSUS CURRENT VALUATION RESIDENTIAL" : "Current Valuation - Residential",
    "Current Valuation - Res" : "Current Valuation - Residential",
    "MUL_CURR_VAL_RES" : "Current Valuation - Residential",
    "CENSUS Current Valuation - Res" : "Current Valuation - Residential",
    "CURRENT VALUATION COMM" : "Current Valuation - Commercial",
    "Current Valuation - Comm" : "Current Valuation - Commercial",
    "Current Valuation Comm" : "Current Valuation - Commercial",
    "MUL_CURR_VAL_COMM" : "Current Valuation - Commercial",
    "CURRENT VALUATION COMMERCIAL": "Current Valuation - Commercial",
    "ADDRESS" : "Address",
    "FULL_ADDRESS" : "Address",
    "ADU/TED ON PERMIT?" : "ADU/TED on Permit?",
    "ADU/TED ON ADDRESS\n" : "ADU or TED on Address?",
    "ADU/TED ON ADDRESS" : "ADU or TED on Address?",
    "ADU/TED ON ADDRESS?" : "ADU or TED on Address?",
    "ADU OR TED ON ADDRESS?" : "ADU or TED on Address?",
    "TOTAL CONSTRUCTION VALUATION" : "Total Construction Valuation",
    "Total Construction Valuation TCV" : "Total Construction Valuation",
    "Total Costruction Valuation" : "Total Construction Valuation",
    "CURRENT MARKET VALUATION" : "Current Market Valuation",
    "Current Market Valuation CMV" : "Current Market Valuation"
    }

In [ ]:
def find_header_row_in_main(file, sheet):
    """
    Finds the row number that likely contains the main header based on >50% non-null values.
    
    Args:
    file: the file path (string) for the development report workbook
    sheet: the specific data dump sheet (string) 

    Outputs:
    header_row: the best row number (index) for the main header row
    """
    # Get the first few rows to see where the header exists
    preview = pd.read_excel(file, sheet, header=None, nrows=5).iloc[:, 0:15]
    total_cols = preview.shape[1]
    
    header_row = None

    # Starting with top row, iterate through until we get columns with names
    for i, row in preview.iterrows():
        non_null_count = row.notna().sum()
        unnamed_count = sum(str(col).startswith("Unnamed") for col in row.astype(str))

        # At least 50% of the row contains valid data and not mostly 'Unnamed'
        if non_null_count > (0.5 * total_cols) and unnamed_count < (0.5 * total_cols):
            header_row = i
            break

    if header_row is None:
        print(f"Warning: Header row for the '{sheet}' main was not found. Please ensure the data is formatted like previous data dump months.")

    return header_row

# Example
file = os.getcwd()+'/data/FY23 Development Report Workbook.xlsx'
sheet = '03 Data Dump'
find_header_row_in_main(file, sheet)

0

In [ ]:
def extract_data_dump_main(file, sheet):
    
    header = find_header_row_in_main(file, sheet)

    # Read Excel with the identified header row
    data = pd.read_excel(file, sheet, header=header).iloc[:,0:15]

    # Rename columns using mapping
    data.rename(columns=column_mapping, inplace=True)
    
    # Filter out rows where 'Record Number' contains numeric values
    data = data[~data['Record Number'].apply(lambda x: isinstance(x, (int, float)))]
    data = data[data['Record Number']!='count']

    # Filter out NaN values in 'Record Number'
    data = data[~data['Record Number'].isna()]

    # Remove only the trailing newline character
    data['ADU or TED on Address?'] = data['ADU or TED on Address?'].str.rstrip('\n').replace('', np.nan)

    # Add 'Property Type' column
    data['Property Type'] = np.nan

    # Keep rows where there has been a valuation change
    data = data.loc[data.groupby(['Record Number','Address'])[data.columns].apply(lambda x: x.notnull().sum(axis=1).idxmax())]

    return data

data_dump_main = extract_data_dump_main(file, sheet)

In [71]:
def find_header_row_in_revisions(file, sheet):
    """
    Finds the row number that likely contains the revisions header based on >50% non-null values.
    
    Args:
    file: the file path (string) for the development report workbook
    sheet: the specific data dump sheet (string) 

    Outputs:
    header_row: the best row number (index) for the revisions header row
    """
    # Get the first few rows to see where the header exists
    preview = pd.read_excel(file, sheet, header=None, nrows=5).iloc[:,16:]
    total_cols = preview.shape[1]
    
    header_row = None

    # Starting with top row, iterate through until we get columns with names
    for i, row in preview.iterrows():
        non_null_count = row.notna().sum()
        unnamed_count = sum(str(col).startswith("Unnamed") for col in row.astype(str))

        # At least 50% of the row contains valid data and not mostly 'Unnamed'
        if non_null_count > (0.5 * total_cols) and unnamed_count < (0.5 * total_cols):
            header_row = i
            break

    if header_row is None:
        print(f"Warning: Header row for the '{sheet}' revisions were not found. Please ensure the data is formatted like previous data dump months.")

    return header_row

# Example
file = os.getcwd()+'/data/FY23 Development Report Workbook.xlsx'
sheet = '03 Data Dump'
find_header_row_in_revisions(file, sheet)

2

In [11]:
def extract_data_dump_revisions(file, sheet):
    
    header = find_header_row_in_revisions(file, sheet)

    # Read Excel with the identified header row
    data = pd.read_excel(file, sheet, header=header).iloc[:,16:]

    # Rename columns using mapping
    data.rename(columns=column_mapping, inplace=True)
    
    # Check if any column matches the column_mapping keys
    if not any(col in data.columns for col in chain(*column_mapping.items())):
        return pd.DataFrame()  # Return an empty DataFrame if no matches

    # Convert relevant columns to numeric
    numeric_cols = ['Project Cost', 'Job Cost Valuation', 'Current Valuation - Commercial', 'Current Valuation - Residential']
    for col in numeric_cols:
        data[col] = pd.to_numeric(data[col], errors='coerce')

    # Filter out rows where 'Record Number' is numeric
    if 'Record Number' in data.columns:
        data = data[~data['Record Number'].apply(lambda x: isinstance(x, (int, float)))]

    # Keep rows where there has been a valuation change
    data = data[(abs(data['Project Cost'].fillna(0)) > 0) | 
                (abs(data['Job Cost Valuation'].fillna(0)) > 0) | 
                (abs(data['Current Valuation - Commercial'].fillna(0)) > 0) | 
                (abs(data['Current Valuation - Residential'].fillna(0)) > 0)]

    # Drop columns with "Unnamed" in their name
    data = data.loc[:, ~data.columns.str.contains("Unnamed", na=False)]

    # Add 'Property Type' column
    data['Property Type'] = 'Modification to Work in Progress'

    # Calculate 'Total Construction Valuation'
    data['Total Construction Valuation'] = data['Project Cost'].fillna(0) + data['Job Cost Valuation'].fillna(0)

    # Calculate 'Current Market Valuation'
    data['Current Market Valuation'] = (data['Current Valuation - Commercial'].fillna(0) + 
                                        data['Current Valuation - Residential'].fillna(0) - 
                                        data['Total Construction Valuation']).clip(lower=0)

    # Remove duplicate rows
    data.drop_duplicates(inplace=True)

    return data

# Example usage
data_dump_revisions = extract_data_dump_revisions(file, sheet)


In [12]:
def combine_main_and_revisions(main, revisions):
    data = pd.concat([main, revisions], ignore_index = True)
    return data

data_dump = combine_main_and_revisions(data_dump_main, data_dump_revisions)

In [13]:
data_dump

,Category,Record Number,Commercial Subtype,Residential Subtype,ADU/TED on Permit?,Work Description,Business Name,Project Cost,Job Cost Valuation,Current Valuation - Commercial,Current Valuation - Residential,Address,ADU or TED on Address?,Total Construction Valuation,Current Market Valuation,Property Type
0,08 - New Business,2019-MSS-COM-00288,BNRHM - New Hotel/Motel/Cabin,NaN,NaN,HOMEWOOD SUITES/Construction of a new 99-guest...,NIELSEN COMMERCIAL INC,NaN,3837055.65,10491011.40,NaN,4823 N RESERVE ST,_x000D_,3837055.65,6653955.75,NaN
1,08 - New Business,2020-MSS-COM-00074,BNCON - New Other,NaN,NaN,HOMEWOOD SUITES PORTE COCHERE/Construction of ...,NIELSEN COMMERCIAL INC,150000.0,0.00,NaN,NaN,4823 N RESERVE ST,_x000D_,150000,0.00,NaN
2,08 - New Business,2020-MSS-COM-00266,BNCSC - New Store/Customer Service,NaN,NaN,OREILLY AUTO PARTS/NEW AUTO PARTS STORE/ IIB/ ...,ZERNCO INC,NaN,393688.68,1446325.44,NaN,2704 W BROADWAY,_x000D_,393688.68,1052636.76,NaN
3,01 - Remodel Commercial,2021-MSS-COM-00221,BAARC - Add/Alter/Remodel Commercial,NaN,NaN,T-MOBILE PROPOSED PROJECT INCLUDES MODIFYING ...,VERTEX TOWER SOLUTIONS INC,20000.0,0.00,NaN,NaN,2409 DEARBORN AVE SHELL,_x000D_,20000,0.00,NaN
4,01 - Remodel Commercial,2021-MSS-COM-00258,BAARC - Add/Alter/Remodel Commercial,NaN,NaN,AUTOMATED LIVING/ Interior only tenant improve...,CARL CONSTRUCTION INC,250000.0,0.00,NaN,NaN,2935 STOCKYARD RD STE N101,_x000D_,250000,0.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,02 - Remodel Residential,2022-MSS-RES-00255,NaN,BRRRS - Reroof or Reside Residential,NaN,SFR / Remove and dump old composition roofing ...,RUSSELL ROOT CONSTRUCTION,9500.0,0.00,NaN,NaN,4325 BARBARA LN,_x000D_,9500,0.00,NaN
101,02 - Remodel Residential,2022-MSS-RES-00260,NaN,BRRRS - Reroof or Reside Residential,NaN,SFR / Remove & Replace 30# Felt / replace with...,RHINO ROOFING,17370.0,0.00,NaN,NaN,101 FAIRVIEW AVE,_x000D_,17370,0.00,NaN
102,02 - Remodel Residential,2022-MSS-RES-00270,NaN,BRRRS - Reroof or Reside Residential,NaN,SFR / VB / R-3 / REMOVE OLD MATERIALS / SYNTHE...,SIMPLY THE BEST ROOFING,12000.0,0.00,NaN,NaN,3345 CATHY CT,_x000D_,12000,0.00,NaN
103,NaN,2021-MSS-RES-00292.01,NaN,NaN,NaN,2/9/22 REVISION TO CHANGE TO SINGLE STORY,MOSTAD CONSTRUCTION INC,NaN,73717.99,NaN,235680.2,3892 FESTUS WAY,NaN,73717.99,161962.21,Modification to Work in Progress


## Data Definitions for Cleaning

In [34]:
def assign_property_type(row):
    
    # New Construction
    
    # Single Dwelling Attached
    if (row['Residential Subtype'] == 'BNSFT - New Single Family Townhouse' or row['Residential Subtype'] == 'BNSFR - New Single Family Residence') and row['ADU or TED on Address?'] == 'SFR-ATT':
        return 'Single Dwelling Attached'
    # Single Dwelling Detached
    elif (row['Residential Subtype'] == 'BNSFT - New Single Family Townhouse' or row['Residential Subtype'] == 'BNSFR - New Single Family Residence') and (row['ADU or TED on Address?'] == 'SFR-DET' or row['ADU or TED on Address?'] == 'TWNHS'):
        return 'Single Dwelling Detached'
    # ADU
    elif (row['Residential Subtype'] == 'BNSFT - New Single Family Townhouse' or row['Residential Subtype'] == 'BNSFR - New Single Family Residence' or row['Residential Subtype'] == 'BAARR -  Add/Alter/Remodel Residential' or (row['Category'] == '06 - New Miscellaneous' and row['Residential Subtype'] == 'BNRDG - New Detached Garage/Carport')) and (row['ADU or TED on Address?'] == 'ADU' or row['ADU/TED on Permit?'] == 'ADU'):
        return 'ADU'
    # Duplex
    elif (row['Category'] == '04 - New Duplex' or((row['Residential Subtype'] == 'BNRDX - New Duplex' or row['Commercial Subtype'] == 'BNMRA - New Multifamily 3-4 Units') and (row['ADU or TED on Address?'] == 'Duplex' or row['ADU or TED on Address?'] == 'DUPLEX'))):
        return 'Duplex'
    # Multi-Dwelling Apartment
    elif (row['Commercial Subtype'] == 'BNMRA - New Multifamily 3-4 Units' or row['Commercial Subtype'] == 'BNMRB - New Multifamily 5+ Units') and row['ADU or TED on Address?'] == 'MFR-APT':
        return 'Multi-Dwelling Apartment'
    # Multi-Dwelling Condo
    elif (row['Commercial Subtype'] == 'BNMRA - New Multifamily 3-4 Units' or row['Commercial Subtype'] == 'BNMRB - New Multifamily 5+ Units') and row['ADU or TED on Address?'] == 'MFR-CONDO':
        return 'Multi-Dwelling Condo'
    # TED Single Dwelling
    elif (row['Residential Subtype'] == 'BNSFT - New Single Family Townhouse' or row['Residential Subtype'] == 'BNSFR - New Single Family Residence') and (row['ADU or TED on Address?'] == 'TED SF' or row['ADU or TED on Address?'] == 'TED-SFR' or row['ADU or TED on Address?'] == 'TED') :
        return 'TED Single Dwelling'
    # TED Two Unit
    elif (row['Residential Subtype'] == 'BNSFT - New Single Family Townhouse' or row['Residential Subtype'] == 'BNSFR - New Single Family Residence') and (row['ADU or TED on Address?'] == 'TED 2U' or row['ADU or TED on Address?'] == 'TED-2U'):
        return 'TED Two Unit'    
    # TED 3+ Unit
    elif (row['Residential Subtype'] == 'BNSFT - New Single Family Townhouse' or row['Residential Subtype'] == 'BNSFR - New Single Family Residence' or row['Residential Subtype'] == 'BNMRA - New Multifamily 3 -4 Units') and row['ADU or TED on Address?'] == 'TED 3+':
        return 'TED 3+'
    # Misc. (Garage, Shed, etc.)
    elif ('CARPORT' in str(row['Work Description']).upper() or row['Residential Subtype'] == 'BNRDA - New Detached Accessory Building' or row['Residential Subtype'] == 'BNRDG - New Detached Garage/Carport' or row['Residential Subtype'] == 'BRFND - New Residential Foundation' or row['Residential Subtype'] == 'BO\\S\\R - Other Residential' or (row['Category'] == '06 - New Miscellaneous' and row['Commercial Subtype'] == 'BNCON -  New Other')):
        return 'Misc. (Garage, Shed, etc.)'
    # Assembly
    elif row['Category'] == '07 - New Assembly':
        return 'Assembly'
    # Business
    elif row['Category'] == '08 - New Business':
        return 'Business'
    # Education (Undefined)
    elif row['Category'] == '09 - New Education':
        return 'Education'
    # Hazardous (Undefined)
    elif row['Category'] == '10 - New Hazardous':
        return 'Hazardous'
    # Institutional (Undefined)
    elif 1 == 2:
        return 'Institutional'
    
    # Addition/Remodel

    # Commercial
    if (row['Category'] == '01 - Remodel Commercial' or (row['Category'] == '08 - New Business' and row['Commercial Subtype'] == 'BNCNB -  New Other Than Building')):
        return 'Commercial'
    # Residential
    if row['Category'] == '02 - Remodel Residential':
        return 'Residential'

    # Revision

    # Modification to Work in Progress
    if row['Property Type'] == 'Modification to Work in Progress':
        return 'Modification to Work in Progress'

    # Flag Unspecified
    else:
        return 'Unspecified'

def assign_project_type(row):
    if (row['Property Type'] == 'Commercial') or (row['Property Type'] == 'Residential') or (row['Property Type'] == 'Modification to Work in Progress'):
        return 'Addition/Remodel'
    else:
        return 'New Construction'
    
def assign_property_type_order(row):
    permit_type_order = {
        "Single Dwelling Attached": 1,
        "Single Dwelling Detached": 2,
        "Duplex": 3,
        "Multi-Dwelling Apt": 4,
        "Multi-Dwelling Condo": 5,
        "TED Single Dwelling": 6,
        "TED Two Unit": 7,
        "TED 3+": 8,
        "Misc. (Garage, Shed, etc.)": 9,
        "Assembly": 10,
        "Business": 11,
        "Education": 12,
        "Hazardous": 13,
        "Institutional": 14,
        "Residential": 15,
        "Commercial": 16,
        "Modification to work in progress": 17
    }
    if row['Property Type'] in permit_type_order:
        return permit_type_order[row['Property Type']]
    else:
        return 99


In [36]:
data_dump['Property Type'] = data_dump.apply(assign_property_type, axis=1)
data_dump['Project Type'] = data_dump.apply(assign_project_type, axis=1)
data_dump['Property Type Order'] = data_dump.apply(assign_property_type_order, axis=1)
data_dump

,Category,Record Number,Commercial Subtype,Residential Subtype,ADU/TED on Permit?,Work Description,Business Name,Project Cost,Job Cost Valuation,Current Valuation - Commercial,Current Valuation - Residential,Address,ADU or TED on Address?,Total Construction Valuation,Current Market Valuation,Property Type,Project Type,Property Type Order
0,08 - New Business,2019-MSS-COM-00288,BNRHM - New Hotel/Motel/Cabin,NaN,NaN,HOMEWOOD SUITES/Construction of a new 99-guest...,NIELSEN COMMERCIAL INC,NaN,3837055.65,10491011.40,NaN,4823 N RESERVE ST,_x000D_,3837055.65,6653955.75,Business,New Construction,11
1,08 - New Business,2020-MSS-COM-00074,BNCON - New Other,NaN,NaN,HOMEWOOD SUITES PORTE COCHERE/Construction of ...,NIELSEN COMMERCIAL INC,150000.0,0.00,NaN,NaN,4823 N RESERVE ST,_x000D_,150000,0.00,Business,New Construction,11
2,08 - New Business,2020-MSS-COM-00266,BNCSC - New Store/Customer Service,NaN,NaN,OREILLY AUTO PARTS/NEW AUTO PARTS STORE/ IIB/ ...,ZERNCO INC,NaN,393688.68,1446325.44,NaN,2704 W BROADWAY,_x000D_,393688.68,1052636.76,Business,New Construction,11
3,01 - Remodel Commercial,2021-MSS-COM-00221,BAARC - Add/Alter/Remodel Commercial,NaN,NaN,T-MOBILE PROPOSED PROJECT INCLUDES MODIFYING ...,VERTEX TOWER SOLUTIONS INC,20000.0,0.00,NaN,NaN,2409 DEARBORN AVE SHELL,_x000D_,20000,0.00,Commercial,Addition/Remodel,16
4,01 - Remodel Commercial,2021-MSS-COM-00258,BAARC - Add/Alter/Remodel Commercial,NaN,NaN,AUTOMATED LIVING/ Interior only tenant improve...,CARL CONSTRUCTION INC,250000.0,0.00,NaN,NaN,2935 STOCKYARD RD STE N101,_x000D_,250000,0.00,Commercial,Addition/Remodel,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,02 - Remodel Residential,2022-MSS-RES-00255,NaN,BRRRS - Reroof or Reside Residential,NaN,SFR / Remove and dump old composition roofing ...,RUSSELL ROOT CONSTRUCTION,9500.0,0.00,NaN,NaN,4325 BARBARA LN,_x000D_,9500,0.00,Residential,Addition/Remodel,15
101,02 - Remodel Residential,2022-MSS-RES-00260,NaN,BRRRS - Reroof or Reside Residential,NaN,SFR / Remove & Replace 30# Felt / replace with...,RHINO ROOFING,17370.0,0.00,NaN,NaN,101 FAIRVIEW AVE,_x000D_,17370,0.00,Residential,Addition/Remodel,15
102,02 - Remodel Residential,2022-MSS-RES-00270,NaN,BRRRS - Reroof or Reside Residential,NaN,SFR / VB / R-3 / REMOVE OLD MATERIALS / SYNTHE...,SIMPLY THE BEST ROOFING,12000.0,0.00,NaN,NaN,3345 CATHY CT,_x000D_,12000,0.00,Residential,Addition/Remodel,15
103,NaN,2021-MSS-RES-00292.01,NaN,NaN,NaN,2/9/22 REVISION TO CHANGE TO SINGLE STORY,MOSTAD CONSTRUCTION INC,NaN,73717.99,NaN,235680.2,3892 FESTUS WAY,NaN,73717.99,161962.21,Modification to Work in Progress,Addition/Remodel,99


# Iterating over Multiple Data Dumps

In [16]:
build_sheet_details_df(file)

,Data Dump,Five Reports,Date,Month Number,Month,Calendar Year,Fiscal Year
0,01 Data Dump,01 Five Reports,2022-01-01,01,January,2022,2022
1,02 Data Dump,02 Five Reports,2022-02-01,02,February,2022,2022
2,03 Data Dump,03 Five Reports,2022-03-01,03,March,2022,2022
3,04 Data Dump,04 Five Reports,2022-04-01,04,April,2022,2022
4,05 Data Dump,05 Five Reports,2022-05-01,05,May,2022,2022
5,06 Data Dump,06 Five Reports,2022-06-01,06,June,2022,2022
6,07 Data Dump,07 Five Reports,2021-07-01,07,July,2021,2022
7,08 Data Dump,08 Five Reports,2021-08-01,08,August,2021,2022
8,09 Data Dump,09 Five Reports,2021-09-01,09,September,2021,2022
9,10 Data Dump,10 Five Reports,2021-10-01,10,October,2021,2022


In [42]:
def process_data_dumps(file):

    sheet_details_df = build_sheet_details_df(file)[['Data Dump', 'Date', 'Month', 'Month Number', 'Calendar Year', 'Fiscal Year']]

    # Get sheets that contain both 'data' and 'dump'
    sheets = [sheet for sheet in pd.ExcelFile(file).sheet_names if all(word in sheet.lower() for word in ['data', 'dump'])]

    # List to store processed data frames
    all_data = []

    for sheet in sheets:
        
        # Print Sheet
        print(sheet)

        # Extract main and revision data
        data_dump_main = extract_data_dump_main(file, sheet)
        data_dump_revisions = extract_data_dump_revisions(file, sheet)

        # Combine main and revision data
        data_dump = combine_main_and_revisions(data_dump_main, data_dump_revisions)

        # Fix some weird carriage returns
        data_dump = data_dump.map(lambda x: x.replace('_x000D_', '') if isinstance(x, str) else x)

        data_dump['Property Type'] = data_dump.apply(assign_property_type, axis=1)
        data_dump['Project Type'] = data_dump.apply(assign_project_type, axis=1)
        data_dump['Property Type Order'] = data_dump.apply(assign_property_type_order, axis=1)

        # Drop columns that are unneeded
        data_dump = data_dump.drop([col for col in ['DATE ISSUED', 'TOTAL MOD'] if col in data_dump.columns], axis=1)
        
        # Identifying duplicates based on "Property Type" and "Work Description"
        condo_dupes = data_dump[data_dump['Property Type'] == 'Multi-Dwelling Condo']

        if not condo_dupes.empty:

            # Group by relevant columns (for example, 'Record Number', 'Property Type', etc.)
            consolidated_condos = condo_dupes.groupby(
                ["Record Number"], as_index=False
            ).agg(
                {   'Category': 'first',
                    'Project Cost': 'sum',
                    'Job Cost Valuation': 'sum',
                    'Current Valuation - Commercial': 'sum',
                    'Current Valuation - Residential': 'sum',
                    'Total Construction Valuation': 'sum',
                    'Current Market Valuation': 'sum',
                    'Address': lambda x: ', '.join(x),  # Concatenate addresses
                    'Commercial Subtype': 'first',
                    'Residential Subtype': 'first',
                    'ADU/TED on Permit?': 'first',
                    'Work Description': 'first',
                    'Business Name': 'first',
                    'ADU or TED on Address?': 'first',
                    'Property Type': 'first',
                    'Project Type': 'first'
                }
            )
            # Remove original duplicate rows
            data_dump = data_dump.drop(condo_dupes.index)
            # Append the consolidated rows back into the DataFrame
            data_dump = pd.concat([data_dump, consolidated_condos], ignore_index=True)

        # Now, we will add the "Units" column based on the existing "Property Type" and "Work Description" columns.
        data_dump['Units'] = 0  # Default value
        
        # Set to 2 if "Property Type" is "Duplex"
        data_dump.loc[data_dump['Property Type'].isin(['Single Dwelling Attached', 'Single Dwelling Detached', 'TED Single Dwelling', 'TED Two Unit', 'TED 3+']), 'Units'] = 1

        # Set to 2 if "Property Type" is "Duplex"
        data_dump.loc[data_dump['Property Type'] == 'Duplex', 'Units'] = 2
    
        # Define a mapping for word-based numbers
        word_to_num = {
            "Tri": 3,
            "Quad": 4,
            "Five": 5,
            "Six": 6
        }

        # For "Multi-Dwelling Apartment" or "Multi-Dwelling Condo", extract the new units from "Work Description"
        for index, row in data_dump.iterrows():
            if row['Property Type'] in ['Multi-Dwelling Apartment', 'Multi-Dwelling Condo']:
                work_description = row['Work Description']
                
                # Search for patterns like "4-Plex" or "3 Units"
                match = re.search(r'(\d+)[- ]*(Plex|Unit|APT)', work_description, re.IGNORECASE)
                if match:
                    data_dump.at[index, 'Units'] = int(match.group(1))
                else:
                    # Check for word-based numbers
                    for word, num in word_to_num.items():
                        if re.search(rf'\b{word}\b', work_description, re.IGNORECASE):  # Ensure whole word match
                            data_dump.at[index, 'Units'] = num
                            break  # Stop after the first match
        
        # Create a new column for the sheet name
        data_dump["Data Dump"] = sheet

        # Left join with sheet_details_df on "Sheet Name"
        data_dump = data_dump.merge(sheet_details_df, on="Data Dump", how="left")

        # Append processed sheet data to the list
        all_data.append(data_dump)

    # Concatenate all processed sheets into a single DataFrame
    final_data_dump = pd.concat(all_data, ignore_index=True) if all_data else pd.DataFrame()

    return final_data_dump

file = os.getcwd()+'/data'+'/FY24 Development Report Workbook.xlsx'

data = process_data_dumps(file)

07 Data Dump
08 Data Dump
09 Data Dump
10 Data Dump
11 Data Dump
12 Data Dump
01 Data Dump
02 Data Dump
03 Data Dump
04 Data Dump
05 Data Dump
06 Data Dump


In [43]:
data.loc[data['Property Type']=='Modification to Work in Progress'].columns

Index(['Category', 'Record Number', 'Commercial Subtype',
       'Residential Subtype', 'ADU/TED on Permit?', 'Work Description',
       'Business Name', 'Project Cost', 'Job Cost Valuation',
       'Current Valuation - Commercial', 'Current Valuation - Residential',
       'Address', 'ADU or TED on Address?', 'Total Construction Valuation',
       'Current Market Valuation', 'Property Type', 'Project Type',
       'Property Type Order', 'Units', 'Data Dump', 'Date', 'Month',
       'Month Number', 'Calendar Year', 'Fiscal Year'],
      dtype='object')

# Process & Store Data Dumps FY25

In [19]:
file = os.getcwd()+'/data/FY25 Development Report Workbook.xlsx'

data = process_data_dumps(file)

data.to_csv('output/PermitDataFY2025.csv', index=False)

07 Data Dump
08 Data Dump
09 Data Dump
10 Data Dump
11 Data Dump
12 Data Dump
01 Data Dump


In [20]:
data.loc[data['Property Type']=="Multi-Dwelling Condo"]

,Category,Record Number,Commercial Subtype,Residential Subtype,ADU/TED on Permit?,Work Description,Business Name,Project Cost,Job Cost Valuation,Current Valuation - Commercial,...,Current Market Valuation,Property Type,Project Type,Units,Data Dump,Date,Month,Month Number,Calendar Year,Fiscal Year
978,05 - New MultiFamily,2023-MSS-COM-00338,BNMRB - New Multifamily 5+ Units,None,None,2021 CODES/BLDG G/NEW 12 UNIT CONDO BLDG/VB/R-2,HEADWATERS CONSTRUCTION COMPANY,0.0,794360.4,0.0,...,1858517.08,Multi-Dwelling Condo,New Construction,12,11 Data Dump,2024-11-01,November,11,2024,2025
979,05 - New MultiFamily,2023-MSS-COM-00339,BNMRB - New Multifamily 5+ Units,None,None,2021 CODES/BLDG H/NEW 15 UNIT CONDO BLDG/VB/R-...,HEADWATERS CONSTRUCTION COMPANY,0.0,875212.8,0.0,...,2047682.56,Multi-Dwelling Condo,New Construction,15,11 Data Dump,2024-11-01,November,11,2024,2025


In [21]:
file = os.getcwd()+'/data/FY24 Development Report Workbook.xlsx'

data = process_data_dumps(file)

data.to_csv('output/PermitDataFY2024.csv', index=False)

07 Data Dump
08 Data Dump
09 Data Dump
10 Data Dump
11 Data Dump
12 Data Dump
01 Data Dump
02 Data Dump
03 Data Dump
04 Data Dump
05 Data Dump
06 Data Dump


In [22]:
file = os.getcwd()+'/data/FY23 Development Report Workbook.xlsx'

data = process_data_dumps(file)

data.to_csv('output/PermitDataFY2023.csv', index=False)

07 Data Dump
08 Data Dump
09 Data Dump
10 Data Dump
11 Data Dump
12 Data Dump
01 Data Dump
02 Data Dump
03 Data Dump
04 Data Dump
05 Data Dump
06 Data Dump


In [23]:
file = os.getcwd()+'/data/FY22 Development Report Workbook.xlsx'

data = process_data_dumps(file)

data.to_csv('output/PermitDataFY2022.csv', index=False)

09 Data Dump
10 Data Dump
11 Data Dump
12 Data Dump
01 Data Dump
03 Data Dump
04 Data Dump
05 Data Dump
06 Data Dump
02 Data Dump
08 Data Dump
07 Data Dump


In [24]:
def process_all_data_dumps(folder):
    folder = os.path.join(os.getcwd(), "data")
    all_data = []

    for file in os.listdir(folder):
        if file.endswith(".xlsx") and not file.startswith("~$"):  # Exclude temp Excel files
            file = os.path.join(folder, file)
            df = process_data_dumps(file)  # Use your existing function
            all_data.append(df)

    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        final_df.to_csv('output/PermitDataAll.csv', index=False)

folder = os.getcwd()+'/data'
process_all_data_dumps(folder)


07 Data Dump
08 Data Dump
09 Data Dump
10 Data Dump
11 Data Dump
12 Data Dump
01 Data Dump
02 Data Dump
03 Data Dump
04 Data Dump
05 Data Dump
06 Data Dump
07 Data Dump
08 Data Dump
09 Data Dump
10 Data Dump
11 Data Dump
12 Data Dump
01 Data Dump
02 Data Dump
03 Data Dump
04 Data Dump
05 Data Dump
06 Data Dump
09 Data Dump
10 Data Dump
11 Data Dump
12 Data Dump
01 Data Dump
03 Data Dump
04 Data Dump
05 Data Dump
06 Data Dump
02 Data Dump
08 Data Dump
07 Data Dump
07 Data Dump
08 Data Dump
09 Data Dump
10 Data Dump
11 Data Dump
12 Data Dump
01 Data Dump


In [25]:
data = pd.read_csv(os.getcwd() + "/output/PermitDataAll.csv")
#Counter(data.loc[data['Month']=='June']['Property Type'])
data.columns

Index(['Category', 'Record Number', 'Commercial Subtype',
       'Residential Subtype', 'ADU/TED on Permit?', 'Work Description',
       'Business Name', 'Project Cost', 'Job Cost Valuation',
       'Current Valuation - Commercial', 'Current Valuation - Residential',
       'Address', 'ADU or TED on Address?', 'Total Construction Valuation',
       'Current Market Valuation', 'Property Type', 'Project Type', 'Units',
       'Data Dump', 'Date', 'Month', 'Month Number', 'Calendar Year',
       'Fiscal Year'],
      dtype='object')

In [26]:
Counter(data['Property Type'])

Counter({'Residential': 2515,
         'Commercial': 860,
         'Single Dwelling Detached': 373,
         'Misc. (Garage, Shed, etc.)': 188,
         'Multi-Dwelling Apartment': 127,
         'Business': 81,
         'Single Dwelling Attached': 79,
         'TED Single Dwelling': 68,
         'Modification to Work in Progress': 66,
         'ADU': 64,
         'TED 3+': 49,
         'TED Two Unit': 45,
         'Duplex': 21,
         'Multi-Dwelling Condo': 3,
         'Education': 2,
         'Hazardous': 1,
         'Assembly': 1})

In [27]:
data.loc[data['Property Type'] == 'Multi-Dwelling Apartment', ['Work Description', 'Units', 'Date']].sort_values(by='Date').to_csv('output/units.csv')


In [28]:
data.loc[data['Property Type'] == 'Multi-Dwelling Condo', ['Work Description', 'Units', 'Date']].sort_values(by='Date')

,Work Description,Units,Date
1485,2021 CODES/NEW 4-PLEX CONDO BLDG/VB/R-2,4,2023-09-01
4383,2021 CODES/BLDG G/NEW 12 UNIT CONDO BLDG/VB/R-2,12,2024-11-01
4384,2021 CODES/BLDG H/NEW 15 UNIT CONDO BLDG/VB/R-...,15,2024-11-01
